In [144]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import re 

In [180]:
print(os.listdir("./input"))

imdb = pd.read_csv('./input/imdb_labelled.txt', sep ='\t', header = None)

imdb.columns = ["sentence",  "sentiment"]

amazon = pd.read_csv('./input/amazon_cells_labelled.txt', sep ='\t', header = None)

amazon.columns = ["sentence",  "sentiment"]

yelp = pd.read_csv('./input/yelp_labelled.txt', sep ='\t', header = None)

yelp.columns = ["sentence",  "sentiment"]





['amazon_cells_labelled.txt', 'imdb_labelled.txt', 'yelp_labelled.txt']


In [173]:


tfidf = TfidfVectorizer()
X = imdb['sentence']
y = imdb['sentiment']
X = tfidf.fit_transform(X)


In [174]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

clf = LinearSVC()

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.88      0.67      0.76        84
           1       0.67      0.88      0.76        66

    accuracy                           0.76       150
   macro avg       0.77      0.77      0.76       150
weighted avg       0.79      0.76      0.76       150



In [178]:
tfidf = TfidfVectorizer()
X = amazon['sentence']
y = amazon['sentiment']
X = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

clf = LinearSVC()

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.78      0.91      0.84        96
           1       0.90      0.77      0.83       104

    accuracy                           0.83       200
   macro avg       0.84      0.84      0.83       200
weighted avg       0.84      0.83      0.83       200



In [181]:
tfidf = TfidfVectorizer()
X = yelp['sentence']
y = yelp['sentiment']
X = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

clf = LinearSVC()

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.84      0.81        97
           1       0.83      0.78      0.80       103

    accuracy                           0.81       200
   macro avg       0.81      0.81      0.80       200
weighted avg       0.81      0.81      0.80       200

